In [1]:
# Notebook setup
import nengo                                # neural network simulation library
from nengo.utils.functions import piecewise # useful for defining a piecewise function of time
import numpy as np                          # a scientific computing library
from matplotlib import pyplot as plt        # a plotting library
from mpl_toolkits.mplot3d import Axes3D     # necessary for generating 3D plots
plt.rcParams['axes.labelsize'] = 20         # set default plot axes label font size
plt.rcParams['axes.titlesize'] = 24         # set default plot title font size
plt.rcParams['legend.fontsize'] = 18        # set default plot legend font size
# make plots inline in the notebook instead of creating a new window
%matplotlib inline

# Use this function when asked to compute the RMSE
def compute_rmse(x, y):
    """Computes the root mean-square error"""
    assert np.array_equal(x.shape, y.shape), (
        'x %s and y %s shapes do not match. Fix them before using compute_rmse.' % 
        (str(x.shape), str(y.shape)))
    ret = np.sqrt(np.mean((x-y)**2))
    return ret

# 2) Transformation
In class we discuss how you can find decoders that approximate arbitrary functions of the input. Here we will look at the properties of decoding and transformation. 

## 2.1) Computing nonlinear functions

### 2.1.1) Feedforward computation
Using nengo, build feedforward networks that compute $x^2$, $x^3$, and $\cos(2\pi x)$. For each network:
- Make a plot of the decoded output. Overlay this plot with the stimulus and desired output.
- Report the RMSE.

Your networks should have two `Ensemble`s, and the function should be computed in the `Connection` between them. Although Nengo permits it, do not compute the function in the `Connection` from the input to the first `Ensemble`.

In [5]:
# Use the following in your simulation
T = 1.               # duration of simulation
tau_ens_probe = .01  # Use this as the synapse parameter when creating Probes of Ensembles
in_fun = lambda t: np.sin(2*np.pi*t)  # input function to your network
N = 100  # Number of neurons in each Ensemble
from numpy import pi 

model1 = nengo.Network()
with model1:
    inp = nengo.Node(in_fun)
    ens1 = nengo.Ensemble(n_neurons=N, 
                    dimensions = 1
                    )
    ens2 = nengo.Ensemble(n_neurons=N, 
                dimensions = 1
                )

    probe = nengo.Probe(ens2, 
                         synapse=tau_ens_probe
                        )
    nengo.Connection(inp, ens1)
    nengo.Connection(ens1, ens2, function = lambda x: x**2)

model2 = nengo.Network()
with model2:
    inp = nengo.Node(in_fun)
    ens1 = nengo.Ensemble(n_neurons=N, 
                    dimensions = 1
                    )
    ens2 = nengo.Ensemble(n_neurons=N, 
                dimensions = 1
                )

    probe = nengo.Probe(ens2, 
                         synapse=tau_ens_probe
                        )
    nengo.Connection(inp, ens1)
    nengo.Connection(ens1, ens2, function = lambda x: x**3)

model3 = nengo.Network()
with model3:
    inp = nengo.Node(in_fun)
    ens1 = nengo.Ensemble(n_neurons=N, 
                    dimensions = 1
                    )
    ens2 = nengo.Ensemble(n_neurons=N, 
                dimensions = 1
                )

    probe = nengo.Probe(ens2, 
                         synapse=tau_ens_probe
                        )
    nengo.Connection(inp, ens1)
    nengo.Connection(ens1, ens2, function = lambda x: np.cos(2*pi*x))

with nengo.Simulator(model1) as sim1: 
    sim1.run(T)

with nengo.Simulator(model2) as sim2: 
    sim2.run(T)

with nengo.Simulator(model3) as sim3: 
    sim3.run(T)
    
plt.figure()
plt.plot(sim1.trange(), sim1.data[probe])

Building finished in 0:00:01.                                                   
Simulating finished in 0:00:01.                                                 
Building finished in 0:00:01.                                                   
Simulating finished in 0:00:01.                                                 
Building finished in 0:00:01.                                                   
Simulating finished in 0:00:01.                                                 


KeyError: <Probe at 0x8b9ae48 of 'decoded_output' of <Ensemble (unlabeled) at 0x8b9ae10>>

### 2.1.2) How does the number of neurons affect the decode quality?
For the network computing $x^2$, vary the number of neurons and report how the RMSE varies with number of neurons.
 - Plot number of neurons vs RMSE on a [semilogx](http://matplotlib.org/api/pyplot_api.html?highlight=semilogx#matplotlib.pyplot.semilogx) plot

In [6]:
sim1.data

{<Probe at 0x8b9a860 of 'decoded_output' of <Ensemble (unlabeled) at 0x28450b8>>: [array([ 0.]), array([ 0.]), array([ 0.04446457]), array([-0.03606947]), array([-0.01891713]), array([ 0.0025058]), array([ 0.00181308]), array([-0.03767021]), array([-0.03901506]), array([ 0.00049002]), array([ 0.00618785]), array([-0.01477537]), array([-0.02029488]), array([-0.04064474]), array([-0.00742501]), array([ 0.00827652]), array([-0.00165726]), array([ 0.00406946]), array([-0.00884374]), array([-0.02663641]), array([-0.01135903]), array([ 0.01439326]), array([ 0.00441858]), array([ 0.00212765]), array([-0.0195465]), array([-0.00832656]), array([ 0.00589332]), array([-0.00104947]), array([-0.00453483]), array([ 0.01372696]), array([ 0.02230107]), array([ 0.00529638]), array([ 0.00140356]), array([-0.01479218]), array([ 0.01267561]), array([ 0.01638611]), array([ 0.04414282]), array([ 0.01913667]), array([ 0.02229101]), array([ 0.0322131]), array([ 0.03729421]), array([ 0.03047681]), array([ 0.04

In [ ]:
# Use the following in your simulation
T = 1.  # duration of simulation
tau_ens_probe = .01  # Use this as the synapse parameter when creating Probes of Ensembles
in_fun = lambda t: np.sin(2*np.pi*t)  # input function to your network
Ns = [10, 20, 50, 100, 200, 500, 1000]  # Number of neurons in each Ensemble to test


# YOUR CODE HERE
raise NotImplementedError()

## 2.2) Network factorization
You'll find that there are multiple ways to build networks that compute the same function. Here we'll build networks that compute

$$x_0x_1+x_2$$

### 2.2.1) Network structure 1
Build a nengo network consisting of a 3D ensemble connected to a 1D ensemble to compute the desired function.
- Make a plot of the input as decoded from the 3D ensemble. Overlay this plot with the actual input.
- Make a plot of the decoded output of the 1D ensemble. Overlay this plot with the desired function output.
- Make a plot of the 1D ensemble decode error (with respect to the desired output) and report the RMSE.

In [ ]:
T = 2.  # duration of simulation
tau_ens_probe = .01  # Use this as the synapse parameter when creating Probes of Ensembles
in_fun = piecewise({0.:(.8, .5, .2),
                    .5:(.8, 0., .2), 
                    1.:(0., -.8, .5)})  # input function to your network
N0 = 100  # Number of neurons in the 3D Ensemble
N1 = 100  # Number of neurons in the 1D Ensemble


# YOUR CODE HERE
raise NotImplementedError()

### 2.2.2) Network structure 2
Now we'll build a network with a different structure to compute the same desired function. For this network:
- Create a 2D ensemble and a 1D ensemble.
- Connect the first two dimensions of the input to the 2D ensemble. (Hint: Nengo supports slice notation)
- Connect the 2D ensemble to the 1D ensemble and compute $x_0x_1$ in this connection.
- Connect the third dimension of the input to the 1D ensemble.

Make the same plots as in 2.2.1 using this network.

In [ ]:
T = 2.  # duration of simulation
tau_ens_probe = .01  # Use this as the synapse parameter when creating Probes of Ensembles
in_fun = piecewise({0.:(.8, .5, .2),
                    .5:(.8, 0., .2), 
                    1.:(0., -.8, .5)})  # input function to your network
N0 = 100  # Number of neurons in the 2D Ensemble
N1 = 100  # Number of neurons in the 1D Ensemble


# YOUR CODE HERE
raise NotImplementedError()

### 2.2.3)
The total number of neurons is the same in the networks of 2.2.1 and 2.2.2. 
Explain the difference in performance.

YOUR ANSWER HERE